In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Load the dataset
df = pd.read_csv("D:/unity_projects/last_wala/ab_toh_final/ab_toh_final/Assets/CarDataLog.csv")

# Check for missing values
print("\nMissing values:\n")
print(df.isnull().sum())


Missing values:

Timestamp         0
Position_X        0
Position_Y        0
Position_Z        0
Rotation_X        0
Rotation_Y        0
Rotation_Z        0
Speed             0
Steering_Input    0
Throttle_Input    0
Brake_Input       0
WheelRPM_FL       0
WheelRPM_FR       0
WheelRPM_RL       0
WheelRPM_RR       0
MotorTorque_FL    0
MotorTorque_FR    0
MotorTorque_RL    0
MotorTorque_RR    0
BrakeTorque_FL    0
BrakeTorque_FR    0
BrakeTorque_RL    0
BrakeTorque_RR    0
Action            0
dtype: int64


In [2]:

# Load dataset
file_path = "D:/unity_projects/last_wala/ab_toh_final/ab_toh_final/Assets/CarDataLog.csv"
data = pd.read_csv(file_path)

# Drop irrelevant columns
drop_cols = ['Timestamp']  # Remove timestamp as it's not useful
data = data.drop(columns=[col for col in drop_cols if col in data.columns])

# Convert 'Action' categorical column to numerical
data['Action'] = data['Action'].astype('category').cat.codes

# Define features (X) and target variables (y)
features = [col for col in data.columns if col not in ['Speed', 'Steering_Input', 'Brake_Input']]
X = data[features]

y_speed = data['Speed']
y_steering = data['Steering_Input']
y_brake = data['Brake_Input']

In [3]:
print(data.columns)
# Check for missing values

Index(['Position_X', 'Position_Y', 'Position_Z', 'Rotation_X', 'Rotation_Y',
       'Rotation_Z', 'Speed', 'Steering_Input', 'Throttle_Input',
       'Brake_Input', 'WheelRPM_FL', 'WheelRPM_FR', 'WheelRPM_RL',
       'WheelRPM_RR', 'MotorTorque_FL', 'MotorTorque_FR', 'MotorTorque_RL',
       'MotorTorque_RR', 'BrakeTorque_FL', 'BrakeTorque_FR', 'BrakeTorque_RL',
       'BrakeTorque_RR', 'Action'],
      dtype='object')


In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
import xgboost as xgb
import lightgbm as lgb



# Split data into train and test sets
X_train, X_test, y_train_speed, y_test_speed = train_test_split(X, y_speed, test_size=0.2, random_state=42)
X_train, X_test, y_train_steering, y_test_steering = train_test_split(X, y_steering, test_size=0.2, random_state=42)
X_train, X_test, y_train_brake, y_test_brake = train_test_split(X, y_brake, test_size=0.2, random_state=42)

# Train XGBoost for Speed
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
xgb_model.fit(X_train, y_train_speed)
y_pred_speed = xgb_model.predict(X_test)

# Train LightGBM for Steering Input
lgb_steering_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
lgb_steering_model.fit(X_train, y_train_steering)
y_pred_steering = lgb_steering_model.predict(X_test)

# Train LightGBM for Brake Input
lgb_brake_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
lgb_brake_model.fit(X_train, y_train_brake)
y_pred_brake = lgb_brake_model.predict(X_test)

# Evaluate models
def evaluate_model(y_true, y_pred, model_name):
    print(f"{model_name} - R² Score: {r2_score(y_true, y_pred):.4f}, MAE: {mean_absolute_error(y_true, y_pred):.4f}")

evaluate_model(y_test_speed, y_pred_speed, "XGBoost (Speed)")
evaluate_model(y_test_steering, y_pred_steering, "LightGBM (Steering)")
evaluate_model(y_test_brake, y_pred_brake, "LightGBM (Brake)")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000590 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3483
[LightGBM] [Info] Number of data points in the train set: 15372, number of used features: 18
[LightGBM] [Info] Start training from score 0.037611
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001746 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 3483
[LightGBM] [Info] Number of data points in the train set: 15372, number of used features: 18
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there

In [5]:
import os
import joblib
# Define export path
export_dir = "D:/unity_projects/ab_toh_final/ab_toh_final/ab_toh_final/Assets/Models"
os.makedirs(export_dir, exist_ok=True)

# Save XGBoost model for Speed
speed_model_path = os.path.join(export_dir, "speed_model.pkl")
joblib.dump(xgb_model, speed_model_path)

# Save LightGBM model for Steering
steering_model_path = os.path.join(export_dir, "steering_model.pkl")
joblib.dump(lgb_steering_model, steering_model_path)

# Save LightGBM model for Brake
brake_model_path = os.path.join(export_dir, "brake_model.pkl")
joblib.dump(lgb_brake_model, brake_model_path)

print("✅ Models saved as .pkl at:", export_dir)


✅ Models saved as .pkl at: D:/unity_projects/ab_toh_final/ab_toh_final/ab_toh_final/Assets/Models


In [6]:
# import os
# import json
# import pandas as pd
# import onnxmltools
# from skl2onnx.common.data_types import FloatTensorType
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import r2_score, mean_absolute_error
# import xgboost as xgb
# import lightgbm as lgb

# # Define Paths
# export_dir = "D:/unity_projects/ab_toh_final/ab_toh_final/ab_toh_final/Assets/Models"
# os.makedirs(export_dir, exist_ok=True)

# # Load data
# data = pd.read_csv("D:/unity_projects/ab_toh_final/ab_toh_final/ab_toh_final/Assets/CarDataLog.csv")

# # Rename features to follow the required pattern (f1, f2, f3, ...)
# original_features = data.columns.tolist()
# renamed_features = {col: f"f{i+1}" for i, col in enumerate(original_features)}
# data.rename(columns=renamed_features, inplace=True)
# data['f24'] = data['f24'].astype('category').cat.codes

# # Define features and targets
# features = list(renamed_features.values())
# y_speed = data['f6']       # Replace 'f6' with the correct column for speed
# y_steering = data['f7']    # Replace 'f7' with the correct column for steering
# y_brake = data['f9']       # Replace 'f9' with the correct column for brake

# # Exclude target features for each model
# X_speed = data.drop(columns=['f6'])       # Exclude speed column
# X_steering = data.drop(columns=['f7'])    # Exclude steering column
# X_brake = data.drop(columns=['f9'])       # Exclude brake column

# # Split data into training and testing sets
# X_train_speed, X_test_speed, y_train_speed, y_test_speed = train_test_split(X_speed, y_speed, test_size=0.2, random_state=42)
# X_train_steering, X_test_steering, y_train_steering, y_test_steering = train_test_split(X_steering, y_steering, test_size=0.2, random_state=42)
# X_train_brake, X_test_brake, y_train_brake, y_test_brake = train_test_split(X_brake, y_brake, test_size=0.2, random_state=42)

# # Train XGBoost for Speed
# xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1)
# xgb_model.fit(X_train_speed, y_train_speed)
# y_pred_speed = xgb_model.predict(X_test_speed)

# # Train LightGBM for Steering Input
# lgb_steering_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
# lgb_steering_model.fit(X_train_steering, y_train_steering)
# y_pred_steering = lgb_steering_model.predict(X_test_steering)

# # Train LightGBM for Brake Input
# lgb_brake_model = lgb.LGBMRegressor(n_estimators=100, learning_rate=0.1)
# lgb_brake_model.fit(X_train_brake, y_train_brake)
# y_pred_brake = lgb_brake_model.predict(X_test_brake)

# # Evaluate models
# def evaluate_model(y_true, y_pred, model_name):
#     print(f"{model_name} - R² Score: {r2_score(y_true, y_pred):.4f}, MAE: {mean_absolute_error(y_true, y_pred):.4f}")

# evaluate_model(y_test_speed, y_pred_speed, "XGBoost (Speed)")
# evaluate_model(y_test_steering, y_pred_steering, "LightGBM (Steering)")
# evaluate_model(y_test_brake, y_pred_brake, "LightGBM (Brake)")

# import joblib
# import os

# # Make sure your models are already trained
# # xgb_model, lgb_steering_model, lgb_brake_model should already exist

